1) Fordward Propagation

- Se consideran todas las columnas menos Death Event para las entradas de la red neuronal.

- Se normalizan los datos con StandardScaler

- Dos capas ocultas, la primera con 10 neuronas y la segunda con 5, Función de activación ReLU.

- 5000 máximo de iteraciones para el entrenamiento

- L = 0.01 tasa de aprendizaje


In [2]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('heart_failure_clinical_records.csv', delimiter=',', decimal='.')
df.columns = ['Age','Anaemia','CPK','Diabetes','Ejection Fraction','High Blood Pressure','Platelets','Serum Creatinine','Serum Sodium','Sex','Smoking','Time','Death Event']

X = df.drop(columns=['Death Event']).values
Y = df['Death Event'].values

# Normalizar los datos
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1/3, random_state=42)

# Número de registros de entrenamiento
n = X_train.shape[0]

np.random.seed(42)
# Inicializar pesos y sesgos para las capas
# Capa 1 -> Capa 2
w_hidden1 = np.random.rand(10, 12)
b_hidden1 = np.random.rand(10, 1)

# Capa 2 -> Capa 3 (nueva capa intermedia)
w_hidden2 = np.random.rand(5, 10)
b_hidden2 = np.random.rand(5, 1)

# Capa 3 -> Salida
w_output = np.random.rand(1, 5)
b_output = np.random.rand(1, 1)

# Funciones de activación
relu = lambda x: np.maximum(x, 0)
relu_derivative = lambda x: (x > 0).astype(float)
logistic = lambda x: 1 / (1 + np.exp(-x))
logistic_derivative = lambda x: logistic(x) * (1 - logistic(x))

# Función de propagación hacia adelante
def forward_prop(X):
    Z1 = w_hidden1 @ X + b_hidden1
    A1 = relu(Z1)
    Z2 = w_hidden2 @ A1 + b_hidden2
    A2 = relu(Z2)
    Z3 = w_output @ A2 + b_output
    A3 = logistic(Z3)
    return Z1, A1, Z2, A2, Z3, A3

# Cálculo de precisión
test_predictions = forward_prop(X_test.transpose())[5] 
test_predictions = (test_predictions >= 0.5).astype(int)  # Convertir las predicciones en valores binarios
accuracy = np.mean(test_predictions == Y_test.reshape(1, -1))  # Calcular la precisión comparando las predicciones con los valores reales
print("ACCURACY: ", accuracy)

#El valor de precisión (accuracy) significa que aproximadamente el 30,65%% de las predicciones 
# realizadas por la red neuronal en el conjunto de prueba (X_test) fueron correctas.

ACCURACY:  0.3065386922615477


In [3]:

L = 0.01  # La tasa de aprendizaje

# Devuelve pendientes para pesos y sesgos
# usando la regla de la cadena
def backward_prop(Z1, A1, Z2, A2, Z3, A3, X, Y):
    dC_dA3 = 2 * (A3 - Y)
    dA3_dZ3 = logistic_derivative(Z3)
    dZ3_dW3 = A2
    dZ3_dA2 = w_output
    dC_dZ3 = dC_dA3 * dA3_dZ3

    dC_dW3 = dC_dZ3 @ dZ3_dW3.T
    dC_dB3 = np.sum(dC_dZ3, axis=1, keepdims=True)

    dC_dA2 = dZ3_dA2.T @ dC_dZ3
    dA2_dZ2 = relu_derivative(Z2)
    dZ2_dW2 = A1
    dZ2_dA1 = w_hidden2
    dC_dZ2 = dC_dA2 * dA2_dZ2

    dC_dW2 = dC_dZ2 @ dZ2_dW2.T
    dC_dB2 = np.sum(dC_dZ2, axis=1, keepdims=True)

    dC_dA1 = dZ2_dA1.T @ dC_dZ2
    dA1_dZ1 = relu_derivative(Z1)
    dZ1_dW1 = X
    dC_dZ1 = dC_dA1 * dA1_dZ1

    dC_dW1 = dC_dZ1 @ dZ1_dW1.T
    dC_dB1 = np.sum(dC_dZ1, axis=1, keepdims=True)

    return dC_dW1, dC_dB1, dC_dW2, dC_dB2, dC_dW3, dC_dB3

# Ejecutar descenso de gradiente
for i in range(100000):
    # seleccionar aleatoriamente un conjunto de datos de entrenamiento
    idx = np.random.choice(n, 1, replace=False)
    X_sample = X_train[idx].transpose()
    Y_sample = Y_train[idx].reshape(1, 1)

    # pasar datos seleccionados aleatoriamente a través de la red neuronal
    Z1, A1, Z2, A2, Z3, A3 = forward_prop(X_sample)

    # distribuir error a través de la retropropagación y devolver pendientes para pesos y sesgos
    dW1, dB1, dW2, dB2, dW3, dB3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, X_sample, Y_sample)

    # Actualizar pesos y sesgos
    w_hidden1 -= L * dW1
    b_hidden1 -= L * dB1
    w_hidden2 -= L * dW2
    b_hidden2 -= L * dB2
    w_output -= L * dW3
    b_output -= L * dB3

# Cálculo de precisión
test_predictions = forward_prop(X_test.transpose())[5]
test_predictions = (test_predictions >= 0.5).astype(int)  # Convertir las predicciones en valores binarios
accuracy = np.mean(test_predictions == Y_test.reshape(1, -1))  # Calcular la precisión comparando las predicciones con los valores reales
print("ACCURACY: ", accuracy)


# El valor de precisión (accuracy) es 94,12% (mayor q el anterior) indica la proporción  de predicciones correctas realizadas 
# por la red neuronal en el conjunto de prueba (X_test) después de entrenarla mediante el descenso de gradiente.

ACCURACY:  0.9412117576484703


2) Fordward Propagation

- Se consideran solo las columnas que me interesan para las entradas de la red neuronal.
Las variables que muestran una correlación moderada con la variable que me interesa Death Event son: Age / Ejection Fraction / Serum Creatinine / Serum.Sodium / Time. De esta forma evitamos la variables Platelets ya que este presentaba mayor cantidad de valores atipicos.

- Se normalizan los datos con StandardScaler

- Dos capas ocultas, la primera con 10 neuronas y la segunda con 5, Función de activación ReLU.

- 5000 máximo de iteraciones para el entrenamiento

- L = 0.01 tasa de aprendizaje

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = ['Age','Ejection Fraction', 'Serum Creatinine','Serum Sodium', 'Time']

X = df[data].values
Y = df['Death Event'].values

# Normalizar los datos
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1/3, random_state=42)

# Número de registros de entrenamiento
n = X_train.shape[0]

np.random.seed(42)
# Inicializar pesos y sesgos para las capas
# Capa 1 -> Capa 2
w_hidden1 = np.random.rand(10, 5)
b_hidden1 = np.random.rand(10, 1)

# Capa 2 -> Capa 3 (nueva capa intermedia)
w_hidden2 = np.random.rand(5, 10)
b_hidden2 = np.random.rand(5, 1)

# Capa 3 -> Salida
w_output = np.random.rand(1, 5)
b_output = np.random.rand(1, 1)

# Funciones de activación
relu = lambda x: np.maximum(x, 0)
relu_derivative = lambda x: (x > 0).astype(float)
logistic = lambda x: 1 / (1 + np.exp(-x))
logistic_derivative = lambda x: logistic(x) * (1 - logistic(x))

# Función de propagación hacia adelante
def forward_prop(X):
    Z1 = w_hidden1 @ X + b_hidden1
    A1 = relu(Z1)
    Z2 = w_hidden2 @ A1 + b_hidden2
    A2 = relu(Z2)
    Z3 = w_output @ A2 + b_output
    A3 = logistic(Z3)
    return Z1, A1, Z2, A2, Z3, A3

# Cálculo de precisión
test_predictions = forward_prop(X_test.transpose())[5] 
test_predictions = (test_predictions >= 0.5).astype(int)  # Convertir las predicciones en valores binarios
accuracy = np.mean(test_predictions == Y_test.reshape(1, -1))  # Calcular la precisión comparando las predicciones con los valores reales
print("ACCURACY: ", accuracy)

#El valor de precisión (accuracy) significa que aproximadamente el 30,65%% de las predicciones 
# realizadas por la red neuronal en el conjunto de prueba (X_test) fueron correctas.

ACCURACY:  0.3065386922615477


In [6]:
L = 0.01  # La tasa de aprendizaje

# Devuelve pendientes para pesos y sesgos
# usando la regla de la cadena
def backward_prop(Z1, A1, Z2, A2, Z3, A3, X, Y):
    dC_dA3 = 2 * (A3 - Y)
    dA3_dZ3 = logistic_derivative(Z3)
    dZ3_dW3 = A2
    dZ3_dA2 = w_output
    dC_dZ3 = dC_dA3 * dA3_dZ3

    dC_dW3 = dC_dZ3 @ dZ3_dW3.T
    dC_dB3 = np.sum(dC_dZ3, axis=1, keepdims=True)

    dC_dA2 = dZ3_dA2.T @ dC_dZ3
    dA2_dZ2 = relu_derivative(Z2)
    dZ2_dW2 = A1
    dZ2_dA1 = w_hidden2
    dC_dZ2 = dC_dA2 * dA2_dZ2

    dC_dW2 = dC_dZ2 @ dZ2_dW2.T
    dC_dB2 = np.sum(dC_dZ2, axis=1, keepdims=True)

    dC_dA1 = dZ2_dA1.T @ dC_dZ2
    dA1_dZ1 = relu_derivative(Z1)
    dZ1_dW1 = X
    dC_dZ1 = dC_dA1 * dA1_dZ1

    dC_dW1 = dC_dZ1 @ dZ1_dW1.T
    dC_dB1 = np.sum(dC_dZ1, axis=1, keepdims=True)

    return dC_dW1, dC_dB1, dC_dW2, dC_dB2, dC_dW3, dC_dB3

# Ejecutar descenso de gradiente
for i in range(100000):
    # seleccionar aleatoriamente un conjunto de datos de entrenamiento
    idx = np.random.choice(n, 1, replace=False)
    X_sample = X_train[idx].transpose()
    Y_sample = Y_train[idx].reshape(1, 1)

    # pasar datos seleccionados aleatoriamente a través de la red neuronal
    Z1, A1, Z2, A2, Z3, A3 = forward_prop(X_sample)

    # distribuir error a través de la retropropagación y devolver pendientes para pesos y sesgos
    dW1, dB1, dW2, dB2, dW3, dB3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, X_sample, Y_sample)

    # Actualizar pesos y sesgos
    w_hidden1 -= L * dW1
    b_hidden1 -= L * dB1
    w_hidden2 -= L * dW2
    b_hidden2 -= L * dB2
    w_output -= L * dW3
    b_output -= L * dB3

# Cálculo de precisión
test_predictions = forward_prop(X_test.transpose())[5]
test_predictions = (test_predictions >= 0.5).astype(int)  # Convertir las predicciones en valores binarios
accuracy = np.mean(test_predictions == Y_test.reshape(1, -1))  # Calcular la precisión comparando las predicciones con los valores reales
print("ACCURACY: ", accuracy)

# El valor de precisión (accuracy) es 94,90% (mayor q el anterior) indica la proporción  de predicciones correctas realizadas 
# por la red neuronal en el conjunto de prueba (X_test) después de entrenarla mediante el descenso de gradiente.

ACCURACY:  0.9490101979604079


3) Fordward Propagation

- Se consideran solo las columnas que me interesan para las entradas de la red neuronal.
Las variables que muestran una correlación moderada con la variable que me interesa Death Event son: Age / Ejection Fraction / Serum Creatinine / Serum.Sodium / Time. De esta forma evitamos la variables Platelets ya que este presentaba mayor cantidad de valores atipicos.

- Normalizo las columnas que no estan entre 0 y 1, dividiendo por el maximo de cada columna.

- Dos capas ocultas, la primera con 10 neuronas y la segunda con 5, Función de activación ReLU.

- 5000 máximo de iteraciones para el entrenamiento

- L = 0.01 tasa de aprendizaje

In [7]:
column_norm= ['Age', 'CPK', 'Ejection Fraction', 'Platelets', 'Serum Creatinine', 'Serum Sodium','Time']

#Normalizó dividiendo cada columna por el maximo valor de esa columna para que de como resultado valores entre 0 y 1
df[column_norm] = df[column_norm] / df[column_norm].max()
data = ['Age','Ejection Fraction', 'Serum Creatinine','Serum Sodium', 'Time'] 
df[data].describe()

,Age,Ejection Fraction,Serum Creatinine,Serum Sodium,Time
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.634618,0.471683,0.145650,0.924380,0.458522
std,0.123129,0.143936,0.107420,0.030164,0.271319
min,0.421053,0.175000,0.053191,0.763514,0.014035
25%,0.526316,0.375000,0.095745,0.905405,0.259649
50%,0.631579,0.475000,0.117021,0.925676,0.396491
75%,0.715789,0.562500,0.148936,0.945946,0.705263
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df[data].values
Y = df['Death Event'].values

# Normalizar los datos
#scaler = StandardScaler()
#X = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1/3, random_state=42)

# Número de registros de entrenamiento
n = X_train.shape[0]

np.random.seed(42)
# Inicializar pesos y sesgos para las capas
# Capa 1 -> Capa 2
w_hidden1 = np.random.rand(10, 5)
b_hidden1 = np.random.rand(10, 1)

# Capa 2 -> Capa 3 (nueva capa intermedia)
w_hidden2 = np.random.rand(5, 10)
b_hidden2 = np.random.rand(5, 1)

# Capa 3 -> Salida
w_output = np.random.rand(1, 5)
b_output = np.random.rand(1, 1)

# Funciones de activación
#relu = lambda x: np.maximum(x, 0)
#logistic = lambda x: 1 / (1 + np.exp(-x))

# Funciones de activación
relu = lambda x: np.maximum(x, 0)
relu_derivative = lambda x: (x > 0).astype(float)
logistic = lambda x: 1 / (1 + np.exp(-x))
logistic_derivative = lambda x: logistic(x) * (1 - logistic(x))

# Función de propagación hacia adelante
def forward_prop(X):
    Z1 = w_hidden1 @ X + b_hidden1
    A1 = relu(Z1)
    Z2 = w_hidden2 @ A1 + b_hidden2
    A2 = relu(Z2)
    Z3 = w_output @ A2 + b_output
    A3 = logistic(Z3)
    return Z1, A1, Z2, A2, Z3, A3

# Cálculo de precisión
test_predictions = forward_prop(X_test.transpose())[5] 
test_predictions = (test_predictions >= 0.5).astype(int)  # Convertir las predicciones en valores binarios
accuracy = np.mean(test_predictions == Y_test.reshape(1, -1))  # Calcular la precisión comparando las predicciones con los valores reales
print("ACCURACY: ", accuracy)

#El valor de precisión (accuracy) significa que aproximadamente el 30,65%% de las predicciones 
# realizadas por la red neuronal en el conjunto de prueba (X_test) fueron correctas.

ACCURACY:  0.3065386922615477


In [9]:
L = 0.01  # La tasa de aprendizaje

# Devuelve pendientes para pesos y sesgos
# usando la regla de la cadena
def backward_prop(Z1, A1, Z2, A2, Z3, A3, X, Y):
    dC_dA3 = 2 * (A3 - Y)
    dA3_dZ3 = logistic_derivative(Z3)
    dZ3_dW3 = A2
    dZ3_dA2 = w_output
    dC_dZ3 = dC_dA3 * dA3_dZ3

    dC_dW3 = dC_dZ3 @ dZ3_dW3.T
    dC_dB3 = np.sum(dC_dZ3, axis=1, keepdims=True)

    dC_dA2 = dZ3_dA2.T @ dC_dZ3
    dA2_dZ2 = relu_derivative(Z2)
    dZ2_dW2 = A1
    dZ2_dA1 = w_hidden2
    dC_dZ2 = dC_dA2 * dA2_dZ2

    dC_dW2 = dC_dZ2 @ dZ2_dW2.T
    dC_dB2 = np.sum(dC_dZ2, axis=1, keepdims=True)

    dC_dA1 = dZ2_dA1.T @ dC_dZ2
    dA1_dZ1 = relu_derivative(Z1)
    dZ1_dW1 = X
    dC_dZ1 = dC_dA1 * dA1_dZ1

    dC_dW1 = dC_dZ1 @ dZ1_dW1.T
    dC_dB1 = np.sum(dC_dZ1, axis=1, keepdims=True)

    return dC_dW1, dC_dB1, dC_dW2, dC_dB2, dC_dW3, dC_dB3

# Ejecutar descenso de gradiente
for i in range(100000):
    # seleccionar aleatoriamente un conjunto de datos de entrenamiento
    idx = np.random.choice(n, 1, replace=False)
    X_sample = X_train[idx].transpose()
    Y_sample = Y_train[idx].reshape(1, 1)

    # pasar datos seleccionados aleatoriamente a través de la red neuronal
    Z1, A1, Z2, A2, Z3, A3 = forward_prop(X_sample)

    # distribuir error a través de la retropropagación y devolver pendientes para pesos y sesgos
    dW1, dB1, dW2, dB2, dW3, dB3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, X_sample, Y_sample)

    # Actualizar pesos y sesgos
    w_hidden1 -= L * dW1
    b_hidden1 -= L * dB1
    w_hidden2 -= L * dW2
    b_hidden2 -= L * dB2
    w_output -= L * dW3
    b_output -= L * dB3

# Cálculo de precisión
test_predictions = forward_prop(X_test.transpose())[5]
test_predictions = (test_predictions >= 0.5).astype(int)  # Convertir las predicciones en valores binarios
accuracy = np.mean(test_predictions == Y_test.reshape(1, -1))  # Calcular la precisión comparando las predicciones con los valores reales
print("ACCURACY: ", accuracy)

# El valor de precisión (accuracy) es 30,65% indica la proporción  de predicciones correctas realizadas 
# por la red neuronal en el conjunto de prueba (X_test) después de entrenarla mediante el descenso de gradiente.
# ¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿Es igual que el anterior porque??????????????????????

ACCURACY:  0.3065386922615477
